# Data from GTEx
Link: https://www.gtexportal.org/home/downloads/adult-gtex/bulk_tissue_expression


**Bulk-Tissue-Expression:**

Contains the gene expression in *healthy* tissue samples from adults. For each gene, the transcription value for each tissue is listed.

**Output file format:**
* id
* sum
* count
* tpm

## Load Data and melt down to long format
→ takes 20 minutes with 3000 rows per chunk

The format of the input file is a column per tissue.
We load the data in chunks and melt it down to a long format with columns for the gene name, tissue and TPM value.


In [1]:
import pandas as pd
import gc
import time

In [2]:
file = "../import_data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct"
output_file = "../processed_data/GTEX_healthy.csv"


# Initialize an output file and overwrite if already exists
with open(output_file, 'w') as f_out:
    f_out.write("Name,Tissue,TPM\n")

In [6]:
# Function to analyze the dataset
def dataset_analysis(df):
    global missing_values, min_tpm, max_tpm
    
    missing_values =+ df.isnull().sum()
     
    if df['TPM'].min() < min_tpm:
        min_tpm = df['TPM'].min()
        
    if df['TPM'].max() > max_tpm:
        max_tpm = df['TPM'].max()

NOTE: 
2000 chunks

3000 chunk = 60 sec → 1000 chunk = 20 sec

4000 chunk = 85 sec → 1000 chunk = 21 sec

In [9]:
missing_values = 0
min_tpm = float('+inf')
max_tpm = float('-inf')

chunksize = 2000

with pd.read_csv(file, delimiter="\t", skiprows=2, chunksize=chunksize) as reader:
    counter = 0
    outputrows = 0

    for df_chunk in reader:
        start_time = time.time() 
        df_melt = pd.melt(df_chunk, id_vars=['Name', 'Description'], var_name='Tissue', value_name='TPM', ignore_index=True)
        df_melt = df_melt.drop(columns=['Description'])
        
        df_melt["TPM"] = df_melt["TPM"].astype(float)
        outputrows += df_melt.shape[0]
        
        dataset_analysis(df_melt)
        
        df_melt.to_csv(output_file, mode='a', header=False, index=False)
        
        # Clear memory
        del df_melt
        gc.collect()
        
        end_time = time.time()
        duration = end_time - start_time        
        print(f"{counter} / 56200 processed in {duration:.2f} seconds")
        
        counter += chunksize
        
print(f"Output file contains {outputrows} rows")

0 / 56200 processed in 51.26 seconds
2000 / 56200 processed in 41.12 seconds
4000 / 56200 processed in 41.12 seconds
6000 / 56200 processed in 39.70 seconds
8000 / 56200 processed in 39.81 seconds
10000 / 56200 processed in 41.38 seconds
12000 / 56200 processed in 39.29 seconds
14000 / 56200 processed in 40.43 seconds
16000 / 56200 processed in 45.23 seconds
18000 / 56200 processed in 40.21 seconds
20000 / 56200 processed in 39.65 seconds
22000 / 56200 processed in 39.84 seconds
24000 / 56200 processed in 39.33 seconds
26000 / 56200 processed in 39.97 seconds
28000 / 56200 processed in 49.78 seconds
30000 / 56200 processed in 46.26 seconds
32000 / 56200 processed in 50.40 seconds
34000 / 56200 processed in 50.00 seconds
36000 / 56200 processed in 46.89 seconds
38000 / 56200 processed in 39.84 seconds
40000 / 56200 processed in 41.31 seconds
42000 / 56200 processed in 40.85 seconds
44000 / 56200 processed in 40.89 seconds
46000 / 56200 processed in 40.19 seconds
48000 / 56200 processed 

### Analyze the dataset

In [12]:
print(f"Missing values:\n"
      f"{missing_values}\n")
print(f"Min TPM: {min_tpm}")
print(f"Max TPM: {max_tpm}")

Missing values:
Name      0
Tissue    0
TPM       0
dtype: int64

Min TPM: 0.0
Max TPM: 747400.0


## Save grouped data

In [26]:
file = "../processed_data/GTEX_healthy.csv"
output_file = "../processed_data/GTEX_healthy_temp.csv"

chunksize = 200000000

# Initialize output file and overwrite if already exists
with open(output_file, 'w') as f_out:
    f_out.write("Name,sum,count\n")
    
with (pd.read_csv(file, chunksize=chunksize) as reader):
    for df_chunk in reader:
        start_time = time.time()
        df_mean = df_chunk.drop(columns=["Tissue"])
        df_mean = df_mean.groupby('Name').agg(['sum','count'])
        df_mean.to_csv(output_file, mode='a', header=False, index=True)
        print(f"{df_mean.shape[0]} rows processed in {duration:.2f} seconds")
              
        del df_mean
        gc.collect()
        
        end_time = time.time()
        duration = end_time - start_time

12000 rows processed in 5.04 seconds
15000 rows processed in 15.53 seconds
15000 rows processed in 14.15 seconds
15000 rows processed in 14.34 seconds
11200 rows processed in 13.95 seconds


## Load Temp file to calculate mean TPM 

In [4]:
df_mean = pd.read_csv("../processed_data/GTEX_healthy_temp.csv")
df_mean.rename(columns={'Name':'id'}, inplace=True)
df_mean['id'] = df_mean['id'].str.split('.').str[0]

df_mean

,Name,sum,count
0,ENSG00000000457.13,1.012564e+05,17382
1,ENSG00000000460.16,4.129176e+04,17382
2,ENSG00000000938.12,7.962391e+05,17382
3,ENSG00000000971.15,1.488672e+06,17382
4,ENSG00000001460.17,5.044793e+04,17382
...,...,...,...
68195,ENSG00000284550.1,5.448620e+01,17382
68196,ENSG00000284553.1,5.785420e+01,11446
68197,ENSG00000284564.1,8.892744e+03,11445
68198,ENSG00000284574.1,3.219920e+03,11445


In [41]:
# group again to combine split gene chunks 
df_mean = df_mean.groupby('id').sum()
df_mean['tpm'] = df_mean['sum']/df_mean['count']
df_mean

,sum,count
id,,
ENSG00000000003,274030.408120,17382
ENSG00000000005,62036.191561,17382
ENSG00000000419,841623.549500,17382
ENSG00000000457,101256.444200,17382
ENSG00000000460,41291.764070,17382
...,...,...
ENSG00000284592,27.276690,17382
ENSG00000284594,88.438900,17382
ENSG00000284595,7720.422600,17382


## Save final file

In [44]:
df_mean = df_mean.drop(columns=['sum', 'count'])
df_mean.to_csv("../processed_data/GTEX_healthy_mean.csv")
print(f'There are {df_mean.shape[0]} genes in the final file')